In [28]:
import nltk # test
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package cess_cat is already up-

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package shake

True

In [29]:
#import data as a dataframe

from pyspark.sql.types import *

file_url = "gs://ishita2/training.1600000.processed.noemoticon.csv"

#set schema 

schema = StructType([
    StructField("label", StringType(), True),    
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("query", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True)
])

twitter_df = spark.read.csv(file_url, header = False, schema = schema)

In [30]:
#get number of partitions 

print(twitter_df.rdd.getNumPartitions())

6


In [31]:
twitter_new = twitter_df.repartition(10)
print(twitter_new.rdd.getNumPartitions())

10


In [32]:
# view first 5 tweet data

twitter_new.take(5)

[Row(label='0', id='2303322829', date='Tue Jun 23 18:01:57 PDT 2009', query='NO_QUERY', user='HunneysRunnin', text="Good night everybody...&quot;what a day...what a day&quot; I'm headed to the gym to do my evening work out. I have to meet my trainer this weekend "),
 Row(label='0', id='2286003454', date='Mon Jun 22 15:52:08 PDT 2009', query='NO_QUERY', user='kurdman', text='@RebazQ i feel you bro, its pretty hot here too! '),
 Row(label='4', id='1468554776', date='Tue Apr 07 02:23:03 PDT 2009', query='NO_QUERY', user='reneasaurus', text="Won my first game of settlers  ..and I didn't even realize I could have won the pound before haha"),
 Row(label='0', id='2259978543', date='Sat Jun 20 18:28:27 PDT 2009', query='NO_QUERY', user='stacmasters', text='Hanging out in the lobby of the hotel at MetroCon. @SnafuComics ditched me for some guest related activities! '),
 Row(label='0', id='2261946759', date='Sat Jun 20 21:37:10 PDT 2009', query='NO_QUERY', user='JackieXTaylor', text="Seriously f

In [33]:
from pyspark.sql.functions import col, lower, regexp_replace, split

#clean the tweet's words for better analysis 
def clean(tweet):
  lower_tweet = lower(tweet)
  clean_tweet = regexp_replace(lower_tweet, "^rt ", "") #remove retweet
  clean_tweet = regexp_replace(clean_tweet, "(https?\://)\S+", "") #remove Links
  clean_tweet = regexp_replace(clean_tweet, "@\w+", "")  # remove @
  clean_tweet = regexp_replace(clean_tweet, "[^a-zA-Z0-9\\s]", "") #remove anything not an alphabetor number
  return clean_tweet

#create new dataframe to hold the cleaned tweets and polarity scores 
clean_tweet_df = twitter_new.select(clean(col("text")).alias("tweet"), col("label").alias("target"))

#print some cleaned tweets 

clean_tweet_df.cache()
clean_tweet_df.show(n=5, truncate=False, vertical=True)



-RECORD 0------------------------------------------------------------------------------------------------------------------------
 tweet  | registered for next year individual tax acct systems and control presentational speaking and consumer behavior oh fun  
 target | 0                                                                                                                      
-RECORD 1------------------------------------------------------------------------------------------------------------------------
 tweet  |  oopsthat was my mistake                                                                                               
 target | 0                                                                                                                      
-RECORD 2------------------------------------------------------------------------------------------------------------------------
 tweet  |  no pandas at the zoo i was going to buy you summat from the gift shop but the c

In [34]:

total_tweets = clean_tweet_df.count()
positive_tweets = clean_tweet_df.filter("label == 4").count()
neutral_tweets = clean_tweet_df.filter("label == 2").count()
negative_tweets = clean_tweet_df.filter("label == 0").count()

print(f"Total Tweets: {total_tweets}")
print(f"Positive Tweets: {positive_tweets}")
print(f"Neutral Tweets: {neutral_tweets}")
print(f"Negative Tweets: {negative_tweets}")

Total Tweets: 1600000
Positive Tweets: 800000
Neutral Tweets: 0
Negative Tweets: 800000


In [35]:
#drop tweets with no text 
clean_tweet_df = clean_tweet_df.replace('', None)

In [36]:
clean_tweet_df = clean_tweet_df.na.drop()

In [37]:
clean_tweet_df.count()

1600000

In [38]:
#shuffle and split the dataset
(train_data, val_data, test_data) = clean_tweet_df.randomSplit([0.7, 0.1, 0.2], seed = 10)

#cache datasets in memory
train_data.cache()
val_data.cache()
test_data.cache()

DataFrame[tweet: string, target: string]

In [39]:
#import the necessary libraries
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit, CrossValidator 

stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()

## Elastic Net Logistic Classifier 

### Using a validator to fine tune hyperparameters

In [40]:
#building a pipeline for validation and model fit
from pyspark.ml.classification import LogisticRegression

tokenizer = RegexTokenizer(inputCol="tweet",
                           outputCol="words",
                           pattern="\\W")
 

stopwords_rm = StopWordsRemover(stopWords=stop_words,
                               caseSensitive=False,
                               inputCol="words",
                               outputCol="filtered")

count_vec = CountVectorizer(minDF=5., vocabSize=2**17,
                           inputCol="filtered",
                           outputCol="vec")
 

sparse_vec = IDF(inputCol='vec', outputCol="features", minDocFreq=10)

label_index = StringIndexer(inputCol = "target", outputCol = "label")


In [41]:
#grid search for the best elastic net parameters 

#select some initial parameters 
lam = 0.02
alpha = 0.3

#build the initial grid model
grid_model = LogisticRegression(labelCol="label", featuresCol="features",
                          maxIter=100, regParam=lam,
                           elasticNetParam=alpha)

#create grid pipeline 
grid_pipline = Pipeline(stages=[tokenizer, stopwords_rm, count_vec, sparse_vec, label_index, grid_model])

#specify a grid map with different model parameters 
grid_builder = ParamGridBuilder().\
    addGrid(grid_model.regParam, [0., 0.01, 0.02, 0.03, 0.04]).\
    addGrid(grid_model.elasticNetParam, [0.,0.1, 0.2, 0.3,0.4,0.5]).\
    build()

#select the evaluator
evaluator = MulticlassClassificationEvaluator()

#put all the parameters together under the validation model
validation = TrainValidationSplit(estimator=grid_pipline,
                           estimatorParamMaps=grid_builder,
                           evaluator=evaluator, 
                           trainRatio=0.8)


In [ ]:
#fit the validation model to validation data 
search_model = validation.fit(val_data)

In [ ]:
#print the parameters of the best fit model 

print("Validated Elastic Net Parameters:",search_model.bestModel.stages[-1]._java_obj.parent().getElasticNetParam())
print("Validated Regular Parameters:",search_model.bestModel.stages[-1]._java_obj.parent().getRegParam())

Validated Elastic Net Parameters: 0.1
Validated Regular Parameters: 0.01


### Fitting the classificaton model with the chosen parameters

In [ ]:
#set the hyper parameters to the hyperparameters selected by validation
best_lambda = search_model.bestModel.stages[-1]._java_obj.parent().getRegParam()
best_alpha = search_model.bestModel.stages[-1]._java_obj.parent().getElasticNetParam()

en_logistic = LogisticRegression(labelCol="label", featuresCol="features",
                          maxIter=100, regParam=best_lambda,
                           elasticNetParam=best_alpha)

pipeline = Pipeline(stages=[tokenizer, stopwords_rm, count_vec, sparse_vec, label_index, en_logistic])



In [ ]:
# fit the model to train data
en_logistic_model = pipeline.fit(train_data)

In [ ]:
#make predictions on test data 
en_logistic_predictions = en_logistic_model.transform(test_data)

In [ ]:
#accuracy measurement 
en_logistic_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

en_logistic_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

logistic_acc = en_logistic_accuracy.evaluate(en_logistic_predictions)
logistic_f1 = en_logistic_f1.evaluate(en_logistic_predictions)

print("Test Accuracy = %g " % (logistic_acc))
print("Test F1 = %g " % (logistic_f1))

Test Accuracy = 0.768668 
Test F1 = 0.768118 


## Decision Tree Classifier 

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

#preprocess the text
tokenizer = RegexTokenizer(inputCol="tweet",
                           outputCol="words",
                           pattern="\\W")

stopwords_rm = StopWordsRemover(stopWords=stop_words,
                                caseSensitive=False,
                                inputCol="words",
                                outputCol="filtered")

count_vec = CountVectorizer(minDF=5., vocabSize=2**17,
                           inputCol="filtered",
                           outputCol="vec")

sparse_vec = IDF(inputCol='vec', outputCol="features", minDocFreq=10)

label_index = StringIndexer(inputCol = "target", outputCol = "label")

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",
                           maxDepth = 20)

#create a pipeline 
pipeline = Pipeline(stages=[tokenizer, stopwords_rm, count_vec, sparse_vec, label_index, dt])




In [ ]:
#fit the model to validation data 
dt_model = pipeline.fit(train_data)

In [ ]:
#make predictions on test data 
dt_predictions = dt_model.transform(test_data)


In [ ]:

#accuracy measurement 
dt_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

dt_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

d_acc = dt_accuracy.evaluate(dt_predictions)
d_f1 = dt_f1.evaluate(dt_predictions)

print("Test Accuracy = %g " % (d_acc))
print("Test F1 = %g " % (d_f1))

Test Accuracy = 0.613927 
Test F1 = 0.572088 


In [ ]:
tree = dt_model.stages[5]
print(tree)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_f7499507803d) of depth 20 with 4177 nodes


## Random Forest Classifier 

In [ ]:
# random forest classifier

from pyspark.ml.classification import RandomForestClassifier

tokenizer = RegexTokenizer(inputCol="tweet",
                           outputCol="words",
                           pattern="\\W")


stopwords_rm = StopWordsRemover(stopWords=stop_words,
                               caseSensitive=False,
                               inputCol="words",
                               outputCol="filtered")

count_vec = CountVectorizer(minDF=5., vocabSize=2**17,
                           inputCol="filtered",
                           outputCol="vec")


sparse_vec = IDF(inputCol='vec', outputCol="features", minDocFreq=10)

label_index = StringIndexer(inputCol = "target", outputCol = "label")


In [ ]:
#grid search for the best parameters

#select some initial parameters 
num_tree = 10
max_depth = 10

#build the initial grid model
grid_model = RandomForestClassifier(labelCol="label",
                                   featuresCol="features",
                                   numTrees=num_tree,
                                    maxDepth=max_depth)
#create grid pipeline 
grid_pipline = Pipeline(stages=[tokenizer, stopwords_rm,
                                count_vec, sparse_vec,
                                label_index, grid_model])
#specify a grid map with different model parameters 
grid_builder = ParamGridBuilder().\
    addGrid(grid_model.numTrees, [50, 100, 150, 200, 250]).\
    addGrid(grid_model.maxDepth, [10, 20, 30]).\
    build()

#select the evaluator
evaluator = MulticlassClassificationEvaluator()

#put all the parameters together under the validation model
validation = TrainValidationSplit(estimator=grid_pipline,
                           estimatorParamMaps=grid_builder,
                           evaluator=evaluator, 
                           trainRatio=0.8)

In [ ]:
#fit the model to validation data 
search_model = validation.fit(val_data)

In [ ]:
#print the parameters of the best fit model 

print("Validated number of trees:",search_model.bestModel.stages[-1]._java_obj.parent().getNumTrees())
print("Validated maximum depth of trees:",search_model.bestModel.stages[-1]._java_obj.parent().getMaxDepth())

Validated number of trees: 200
Validated maximum depth of trees: 30


In [ ]:
#set the hyper parameters to the hyperparameters selected by validation

best_numTree = search_model.bestModel.\
               stages[-1]._java_obj.parent().getNumTrees()
best_maxDepth = search_model.bestModel.\
                stages[-1]._java_obj.parent().getMaxDepth()

rd_forest = RandomForestClassifier(labelCol="label",
                                   featuresCol="features",
                                   numTrees=best_numTree,
                                    maxDepth=best_maxDepth)

pipeline = Pipeline(stages=[tokenizer, stopwords_rm,
                            count_vec, sparse_vec,
                            label_index, rd_forest])

In [ ]:
# fit the model to train data
rd_forest_model = pipeline.fit(train_data)

In [ ]:
#make predictions on test data 
rd_forest_predictions = rd_forest_model.transform(test_data)

In [ ]:

#accuracy measurement 
rd_forest_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

rd_forest_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

forest_acc = rd_forest_accuracy.evaluate(rd_forest_predictions)
forest_f1 = rd_forest_f1.evaluate(rd_forest_predictions)

print("Test Accuracy = %g " % (forest_acc))
print("Test F1 = %g " % (forest_f1))

Test Accuracy = 0.732724 
Test F1 = 0.73118 


In [ ]:
forest = rd_forest_model.stages[5]
print(forest)

RandomForestClassificationModel (uid=RandomForestClassifier_c2c1b2705c17) with 250 trees


## Gradient Boost Tree Classifier

In [21]:
# GBT classifier 
from pyspark.ml.classification import GBTClassifier

tokenizer = RegexTokenizer(inputCol="tweet",
                           outputCol="words",
                           pattern="\\W")

stopwords_rm = StopWordsRemover(stopWords=stop_words,
                               caseSensitive=False,
                               inputCol="words",
                               outputCol="filtered")

count_vec = CountVectorizer(minDF=5., vocabSize=2**17,
                           inputCol="filtered",
                           outputCol="vec")

sparse_vec = IDF(inputCol='vec', outputCol="features", minDocFreq=10)

label_index = StringIndexer(inputCol = "target", outputCol = "label")


In [22]:
#build the pipeline to fit the model 
num_tree = 20

gbt_cl = GBTClassifier(labelCol="label", featuresCol="features",
                            maxIter=num_tree)

pipeline = Pipeline(stages=[tokenizer, stopwords_rm,
                            count_vec, sparse_vec,
                            label_index, gbt_cl])

In [ ]:
# fit the model to train data
gbt_cl_model = pipeline.fit(train_data)

In [ ]:
#make predictions on test data 
gbt_cl_predictions = gbt_cl_model.transform(test_data)

In [ ]:
#accuracy measurement 
gbt_cl_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

gbt_cl_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

gbt_acc = gbt_cl_accuracy.evaluate(gbt_cl_predictions)
gbt_f1 = gbt_cl_f1.evaluate(gbt_cl_predictions)

print("Test Accuracy = %g " % (gbt_acc))
print("Test F1 = %g " % (gbt_f1))


Test Accuracy = 0.659628 
Test F1 = 0.646213 


In [ ]:
gbt_forest = gbt_cl_model.stages[5]
print(gbt_forest)

GBTClassificationModel (uid=GBTClassifier_a246aac7b644) with 20 trees


## Final Results 

In [ ]:

print("Test Elastic Net Logistic Classifier Accuracy = %g " % (logistic_acc))
print("Test Elastic Net Logistic Classifier F1 = %g " % (logistic_f1))

print("Test Decision Tree Classifier Accuracy = %g " % (d_acc))
print("Test Decision Tree Classifier F1 = %g " % (d_f1))

print("Test Random Forest Classifier Accuracy = %g " % (forest_acc))
print("Test Random Forest Classifier F1 = %g " % (forest_f1))



Test Elastic Net Logistic Classifier Accuracy = 0.768668 
Test Elastic Net Logistic Classifier F1 = 0.768118 
Test Decision Tree Classifier Accuracy = 0.613927 
Test Decision Tree Classifier F1 = 0.572088 
Test Random Forest Classifier Accuracy = 0.732724 
Test Random Forest Classifier F1 = 0.73118 


In [ ]:
print("Test Gradient Boosted Tree Classifier Accuracy = %g " % (gbt_acc))
print("Test Gradient Boosted Tree F1 = %g " % (gbt_f1))

Test Gradient Boosted Tree Classifier Accuracy = 0.659628 
Test Gradient Boosted Tree F1 = 0.646213 
